In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("oxygen")

create_database('blp_dataset')
create_table('products_tmp_oxygen')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\item")
    folder_image = Path(f"{fo}")

    for file in os.listdir(folder):
        file_path = Path(f"{folder}\{file}\details.html_tab=item-description")

        if file_path.exists():
            page_content = BeautifulSoup(open(Path(f"{file_path}"), encoding='utf8').read())
            
            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.find("pre")

                if desc_content is not None:
                    description = get_content(desc_content)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("h2")

                if raw_title is not None:
                    title = get_content(raw_title)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_oxygen (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                    values = (title, description, "Oxygen", "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("div", attrs = {"class": "panel-body"})
                    img = ""

                    if img_content is not None:
                        img_a = img_content.find("a")
                        
                        if img_a is not None:
                            img = img_a["href"]
                        
                    
                    if img is not "":
                        url_image = Path(f"{folder_image}\{img}")
                        
                        try:
                            # copy images to new directory, renaming with product id
                            if url_image.exists():
                                dst_dir = Path(f"D:\images\_oxygen\{product_id}.jpeg")
                                copyfile(url_image, dst_dir)
                        except:
                            print()
        else:
            file_path = Path(f"{folder}\{file}\details.html_tab=item-details")

            if file_path.exists():
                page_content = BeautifulSoup(open(Path(f"{file_path}"), encoding='utf8').read())

                ## GET IMAGE
                img_content = page_content.find("div", attrs = {"class": "panel-body"})
                img = ""

                if img_content is not None:
                    img_a = img_content.find("a")

                    if img_a is not None:
                        img = img_a["href"]

                if img is not "":
                    url_image = Path(f"{folder_image}\{img}")

                    try:
                        # copy images to new directory, renaming with product id
                        if url_image.exists():
                            dst_dir = Path(f"D:\images\_oxygen\{product_id}_1.jpeg")
                            copyfile(url_image, dst_dir)
                            
                            print(dst_dir)
                    except:
                        print("second_deu_ruim")